In [2]:
import socket
import socketserver
import threading
import uuid
import json
import time
from typing import Tuple

In [3]:
USERNAME="admin"
PASSWORD="admin"
SESSION=""

In [28]:
class ProtocolHandler(socketserver.StreamRequestHandler):
    def authentication(self, username, password)->Tuple[bool, str]:
        if username==USERNAME and password==PASSWORD:
            SESSION=str(uuid.uuid4())
            return (True, SESSION)
        else:
            SESSION=""
            return (False, SESSION)
    
    def say(self, session, content)->bool:
        if session==SESSION:
            print("{} said: {}", USERNAME, content)
            return True
        return False

    def handle(self) -> None:
        self.data=self.rfile.readline().strip()
        print("{} wrote: {}\n".format(self.client_address, self.data))
        request=json.loads(self.data)

        if request["action"]=="login":
            (success, session)=self.authentication(request["authentication"]["username"], request["authentication"]["password"])
            if success:
                SESSION=session
                payload=json.dumps({"action":"authenticated", "session":session, "timestamp":int(time.time())})
                self.wfile.write(bytes(payload, "utf8"))
            else:
                payload=json.dumps({"action":"not_authenticated", "timestamp":int(time.time())})
                self.wfile.write(bytes(payload, "utf8"))
                
        if request["action"]=="say":
            success=self.say(request["session"], request["content"])
            if success:
                payload=json.dumps({"action":"said", "timestamp":int(time.time())})
                self.wfile.write(bytes(payload, "utf8"))
            else:
                payload=json.dumps({"action":"not_authenticated", "timestamp":int(time.time())})
                self.wfile.write(bytes(payload, "utf8"))

In [5]:
class ThreadedTCPServer(socketserver.ThreadingMixIn, socketserver.TCPServer):
    pass

In [29]:
HOST, PORT = "localhost", 10330

# Create the server, binding to localhost on port 9999
with socketserver.TCPServer((HOST, PORT), ProtocolHandler) as server:
    # Activate the server; this will keep running until you
    # interrupt the program with Ctrl-C
    server.serve_forever()

('127.0.0.1', 61015) wrote: b'{"action": "login", "authentication": {"username": "admin", "password": "admin"}, "timestamp": 1634088238}'

dda01d2a-9986-4640-a771-e9b95c929aca
('127.0.0.1', 61044) wrote: b'{"action": "say", "session": "dda01d2a-9986-4640-a771-e9b95c929aca", "content": "Hello", "timestamp": 1634088254}'

('127.0.0.1', 61056) wrote: b'{"action": "say", "session": "dda01d2a-9986-4640-a771-e9b95c929aca", "content": "Hello", "timestamp": 1634088259}'



KeyboardInterrupt: 